In [4]:
from huggingface_hub import login

# Paste your token here
login("..................")


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "mistralai/Mistral-7B-v0.1"  # Base model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
system_prompt = (
    "You are a professional Health Insurance Assistant AI trained to help users understand "
    "everything related to health insurance. You provide accurate, unbiased, and easy-to-understand "
    "answers based on general insurance principles and common healthcare policies.\n\n"
    "Your role is to answer a wide range of questions including:\n"
    "- What health insurance is and why it is important\n"
    "- How to buy and apply for a health insurance policy\n"
    "- What types of health insurance plans are available\n"
    "- Policy coverage, premium, claims, benefits, limitations, and exclusions\n"
    "- Critical illness, emergency coverage, hospitalization, and family add-ons\n"
    "- Frequently asked questions by new customers and policyholders\n\n"
    "You must:\n"
    "- Avoid recommending specific companies or branded policies\n"
    "- Be neutral, informative, and trustworthy\n"
    "- Answer in simple, helpful language for the general public\n"
    "- Use real-world examples only when necessary and relevant\n"
    "- Keep responses short, polite, and informative\n\n"
    "Begin the conversation when the user asks a health insurance question."
)

chat_history = system_prompt


In [7]:
def get_health_insurance_answers(questions):
    answers = []
    for question in questions:
        # Append the question to the chat history
        chat_history = system_prompt + f"User: {question}\nAI:"

        # Tokenize and generate response
        inputs = tokenizer(chat_history, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the output and extract the model's response
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        ai_reply = full_output[len(chat_history):].split("User:")[0].strip()

        # Store the answer
        answers.append(ai_reply)

    return answers


In [8]:
questions = [
    "How do I make a claim?",
    "How Does Private Health Insurance Work?",
    "What Does Private Health Insurance Cover?" ,
    "Will My Health Insurance Plan Save Me Money If I’m Healthy?",
    "Will My Health Insurance Plan Be Affordable If I’m Ill?",
    "What Will My Potential Health Insurance Plan’s Prescription Drug Coverage Be Like?",
    "What Is a Pre-Existing Condition in Health Insurance?",
    "how  long  does  it  take  to  claim  a  life  insurance  policy?",
    "can i claim back critical illness insurance?",
    "how long does it take metlife to process a life insurance claim?",
    "how long does life insurance claim take?",
    "how to claim life insurance money?",
    "can you claim disability insurance premiums on taxes?",
    "can a life insurance claim be denied?"
]

# Get answers
answers = get_health_insurance_answers(questions)

# Print answers
for question, answer in zip(questions, answers):
    print(f"Q: {question}\nA: {answer}\n")


Q: How do I make a claim?
A: Sure. You can make a claim by sending the required documents to your insurance provider. Depending on the policy, the documents could include a doctor's report, bill receipts, and other documents.

Q: How Does Private Health Insurance Work?
A: Private health insurance helps cover the costs of medical treatments and procedures. It can cover outpatient services, inpatient hospital stays, prescription drugs, and more. You can purchase a policy from a private insurer or through your employer.

Q: What Does Private Health Insurance Cover?
A: Private health insurance is a policy that covers medical expenses and can help you pay for hospitalization, doctor's visits, medications, and other healthcare services. It is different from government health insurance, which is provided by the government and is usually free of cost. Private health insurance is usually offered by insurance companies and is bought by individuals or families. It can cover a wide range of health